## Assignment Week 3

## Part 1

### Scrap Toronto postcode data from the Wikipedia page provided using BeautifulSoup

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Get HTML page with the desired data from wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# Create BeaultifulSoup object
soup = BeautifulSoup(source, "html.parser")
                     
#print(soup.prettify())

In [3]:
# Find the desired data under class Wikitable Sortable.
wiki_table = soup.find('table', {'class':'wikitable sortable'})

#print(wiki_table.prettify())

In [4]:
wiki_table_rows = wiki_table.findAll('tr')

#print(wiki_table_rows)

In [5]:
'''
# Create Pandas DataFrame with wiki_table data

res = []
for tr in wiki_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
# Set Pandas DataFrame columns names
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighborhood"])
df
'''

# Create Pandas DataFrame with wiki_table data
res = []
for tr in wiki_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        if (row[1]!='Not assigned'):
            if (row[2]=='Not assigned'):
                row[2]=row[1]
            res.append(row)

for tr in wiki_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        if (row[1]!='Not assigned'):
            if (row[2]=='Not assigned'):
                row[2]=row[1]
            for j in range(0, len(res)):
                if (row[0]==res[j][0]):
                    if (row[2]!=res[j][2]):
                        if row[2] not in res[j][2]:
                            res[j][2] = res[j][2]+', '+row[2]
                        
# Set Pandas DataFrame columns names
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighborhood"])

# Drop duplicate rows:
df = df.drop_duplicates(subset='Postcode', keep='first', inplace=False)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


In [6]:
df.shape

(103, 3)

## Part 2

### "Append" Latitude and Longitune columns to our DataFrame (with data from geocoder)

In [11]:
import geocoder

# iterate through 'res' array (postcode data) and find coordinates for each row
for j in range(0, len(res)):
    print('{0},{1}'.format(j, res[j][0]))
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(res[j][0]))
        lat_lng_coords = g.latlng
    
    # append coordinates to 'res' array
    res[j].append(lat_lng_coords[0])
    res[j].append(lat_lng_coords[1])
    
# convert res array to pandas dataframe
# set columns names
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighborhood", "Latitude", "Longitude"])
# drop duplicate rows:
df = df.drop_duplicates(subset='Postcode', keep='first', inplace=False)
df

0,M3A
1,M4A
2,M5A
3,M5A
4,M6A
5,M6A
6,M7A
7,M9A
8,M1B
9,M1B
10,M3B
11,M4B
12,M4B
13,M5B
14,M5B
15,M6B
16,M9B
17,M9B
18,M9B
19,M9B
20,M9B
21,M1C
22,M1C
23,M1C
24,M3C
25,M3C
26,M4C
27,M5C
28,M6C
29,M9C
30,M9C
31,M9C
32,M9C
33,M1E
34,M1E
35,M1E
36,M4E
37,M5E
38,M6E
39,M1G
40,M4G
41,M5G
42,M6G
43,M1H
44,M2H
45,M3H
46,M3H
47,M3H
48,M4H
49,M5H
50,M5H
51,M5H
52,M6H
53,M6H
54,M1J
55,M2J
56,M2J
57,M2J
58,M3J
59,M3J
60,M4J
61,M5J
62,M5J
63,M5J
64,M6J
65,M6J
66,M1K
67,M1K
68,M1K
69,M2K
70,M3K
71,M3K
72,M4K
73,M4K
74,M5K
75,M5K
76,M6K
77,M6K
78,M6K
79,M1L
80,M1L
81,M1L
82,M2L
83,M2L
84,M3L
85,M4L
86,M4L
87,M5L
88,M5L
89,M6L
90,M6L
91,M6L
92,M9L
93,M1M
94,M1M
95,M1M
96,M2M
97,M2M
98,M3M
99,M4M
100,M5M
101,M5M
102,M6M
103,M6M
104,M6M
105,M6M
106,M9M
107,M9M
108,M1N
109,M1N
110,M2N
111,M3N
112,M4N
113,M5N
114,M6N
115,M6N
116,M9N
117,M1P
118,M1P
119,M1P
120,M2P
121,M4P
122,M5P
123,M5P
124,M6P
125,M6P
126,M9P
127,M1R
128,M1R
129,M2R
130,M4R
131,M5R
132,M5R
133,M5R
134,M6R
135,M6R
136,M7R
137,M9R
138,M9

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
10,M3B,North York,Don Mills North,43.745906,-79.352188
11,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
13,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Part 3

### Import Libraries

In [13]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Filter our DataFrame - We will only work with Boroughs with 'Toronto' in their names.

In [19]:
df2 = df.copy()
df_filtered = df2.query("Borough=='Downtown Toronto' or Borough=='East Toronto' or Borough=='West Toronto' or Borough=='Central Toronto'").reset_index(drop=True)
df_filtered

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


### Create Folium Map of Toronto

In [36]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.665860, -79.383160], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_filtered['Latitude'], df_filtered['Longitude'], df_filtered['Neighborhood']):
    label = folium.Popup()
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Using Foursquare API to explore and segment the neighborhoods.

#### getNearbyVenues() is a function made to get the top 100 venues that are in each neighborhood within a radius of 500 meters.
#### The venue data is stored in the toronto_venues DataFrame.

In [30]:
CLIENT_ID = 'QXLKOWPH4O3GJ0LX43FYI0FPVFT2GXNRVWPPMWS2CMD3VJRX' # your Foursquare ID
CLIENT_SECRET = 'LQVBRFNU1O5HKAHAXMKLRGIRFJ3GY2YZKDBGWO25XCPVB122' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=df_filtered['Neighborhood'],
                                 latitudes=df_filtered['Latitude'],
                                 longitudes=df_filtered['Longitude'])
toronto_venues.head()

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


In [33]:
# Let's check the size of the resulting dataframe
print(toronto_venues.shape)

(1168, 7)


In [34]:
# Let's check how many venues were found for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,16,16,16,16,16,16


In [35]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.
